<a href="https://colab.research.google.com/github/AvRob0/Avrob0.github.io/blob/main/RobinsonAveryFinalProjectEnd_To_End.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imported librarys needed for End-to-end implementation of a Data Science Project
import pandas as pd
import numpy as np
# Loaded the dataset into the hotel_df variable
# and selected all the columns to see how many there are
hotel_df = pd.read_csv('hotel_bookings.csv')
hotel_df.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date'],
      dtype='object')

In [ ]:
hotel_df.info()
# Getting information about the dataset
# Getting the info of the dataset gives me the Range of the Index which is 119390
# It also gives me the total number of coloumns that are in the dataset and gives me all the columns names
# With their non-Null count and datatype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36553 entries, 0 to 36552
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           36553 non-null  object 
 1   is_canceled                     36553 non-null  int64  
 2   lead_time                       36553 non-null  int64  
 3   arrival_date_year               36553 non-null  int64  
 4   arrival_date_month              36553 non-null  object 
 5   arrival_date_week_number        36553 non-null  int64  
 6   arrival_date_day_of_month       36553 non-null  int64  
 7   stays_in_weekend_nights         36553 non-null  int64  
 8   stays_in_week_nights            36553 non-null  int64  
 9   adults                          36553 non-null  int64  
 10  children                        36553 non-null  int64  
 11  babies                          36553 non-null  int64  
 12  meal                            

In [ ]:
hotel_df.isna().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [ ]:
hotel_df.shape

(36553, 32)

In [ ]:
for i in hotel_df.columns:
    #print(i)
    #print(hotel_df[i].unique())
    if hotel_df[i].isna().sum() != 0:
      print('null values in', i, 'column', hotel_df[i].isna().sum())

null values in country column 464
null values in agent column 7851
null values in company column 33550
null values in days_in_waiting_list column 1
null values in customer_type column 1
null values in adr column 1
null values in required_car_parking_spaces column 1
null values in total_of_special_requests column 1
null values in reservation_status column 1
null values in reservation_status_date column 1


In [ ]:
# drop the company column since most of the values are 0
hotel_df = hotel_df.drop(columns = 'company')

# drops 4 rows all containing 0 value in the children column
hotel_df = hotel_df.dropna(subset=['children'])
#hotel_df
# filling the missing values in country column with the frequent mode value
hotel_df['country'].fillna(hotel_df['country'].mode()[0], inplace=True)

# then fill all the missing values in the agent column and use the value used for agent id.
hotel_df['agent'].fillna(0, inplace=True)


In [ ]:
# to replace TA\TO with undefined in distribution_channel column
hotel_df['distribution_channel'].replace('TA/TO', 'Undefined', inplace=True)

#replace Undefined, BB, FB, HB, SC to its meaning.
hotel_df['meal'].replace(['Undefined', 'BB', 'FB', 'HB','SC',], ['no meal', 'full board', 'half board', 'no meal', 'no meal'] , inplace=True)
#inplace = true -> directly modify the dataframe.


In [ ]:
# changing the children and agent column data type into a integer
hotel_df['children'].astype(int)
hotel_df['agent'].astype(int)

# turns the column data type into date and time
pd.to_datetime(hotel_df['reservation_status_date'])

0       2015-07-01
1       2015-07-01
2       2015-07-02
3       2015-07-02
4       2015-07-03
           ...    
36548   2017-05-24
36549   2017-05-24
36550   2017-05-25
36551   2017-05-24
36552          NaT
Name: reservation_status_date, Length: 36553, dtype: datetime64[ns]

In [ ]:
hotel_df.duplicated().sum()
# We hae 5959 of duplicate rows
# Getting rid of the duplicates
hotel_df.drop_duplicates(inplace=True)

In [14]:
#create total night column
hotel_df['total_nights'] = hotel_df['stays_in_weekend_nights'] + hotel_df['stays_in_week_nights']
#convert month name to number then create new arrival date column by combining year month date
hotel_df['arrival_date_month'] = pd.to_datetime(hotel_df['arrival_date_month'], format='%B').dt.month
hotel_df['date_arrival'] = pd.to_datetime(dict(year=hotel_df.arrival_date_year, month=hotel_df.arrival_date_month, day=hotel_df.arrival_date_day_of_month))

In [18]:
# Now I am going to drop the unnecessary columns
drop_col = ['stays_in_weekend_nights','stays_in_week_nights', 'booking_changes','deposit_type','adr','arrival_date_year','arrival_date_month','arrival_date_day_of_month','arrival_date_week_number']

existing_columns = [col for col in drop_col if col in hotel_df.columns]

hotel_df.drop(columns=existing_columns, inplace=True)

hotel_df.describe()


,is_canceled,lead_time,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,agent,days_in_waiting_list,required_car_parking_spaces,total_of_special_requests,total_nights,date_arrival
count,30594.000000,30594.000000,30594.000000,30594.000000,30594.000000,30594.000000,30594.000000,30594.000000,30594.000000,30593.000000,30593.000000,30593.000000,30594.000000,30594
mean,0.260639,79.477545,1.862326,0.141989,0.015689,0.054847,0.024417,0.190952,173.803426,0.348446,0.156801,0.643644,4.312872,2016-07-18 23:33:04.624435968
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2015-07-01 00:00:00
25%,0.000000,7.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,26.000000,0.000000,0.000000,0.000000,2.000000,2016-02-03 00:00:00
50%,0.000000,44.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,240.000000,0.000000,0.000000,0.000000,4.000000,2016-07-25 00:00:00
75%,1.000000,128.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,240.000000,0.000000,0.000000,1.000000,7.000000,2017-01-17 00:00:00
max,1.000000,737.000000,55.000000,10.000000,2.000000,1.000000,26.000000,30.000000,531.000000,185.000000,8.000000,5.000000,69.000000,2017-08-31 00:00:00
std,0.438991,89.713085,0.766202,0.467182,0.126100,0.227686,0.382620,1.142489,114.373861,6.141702,0.370564,0.806716,3.446540,NaN
